In [12]:
import numpy as np
import thinkdsp
%matplotlib inline

In [13]:
num_bands = 64
num_channels = 1024

In [14]:
full_input_wave = thinkdsp.read_wave('flesh_wound.wav')
full_input_spec = full_input_wave.make_spectrum()
full_input_spec.low_pass(5000)
full_input_wave = full_input_spec.make_wave()
full_saw = thinkdsp.SawtoothSignal(freq=240)
full_saw_wave = full_saw.make_wave(duration=full_input_wave.duration, framerate=full_input_wave.framerate)
full_saw_spec = full_saw_wave.make_spectrum()
full_saw_wave = full_saw_spec.make_wave()

In [15]:
full_input_wave.make_audio()

In [16]:
saw_segments = []
for i in np.arange(0, full_saw_wave.duration, full_saw_wave.duration/num_channels):
    saw_segments.append(full_saw_wave.segment(start=i, duration=full_saw_wave.duration/num_channels))
input_segments = []
for i in np.arange(0, full_input_wave.duration, full_saw_wave.duration/num_channels):
    input_segments.append(full_input_wave.segment(start=i, duration=full_saw_wave.duration/num_channels))

In [17]:
temp_final = []
for j in range(num_channels):
    saw_spec = saw_segments[j].make_spectrum()
    input_spec = input_segments[j].make_spectrum()

    input_hs = input_spec.hs
    saw_hs = saw_spec.hs

    saw_bands = np.array_split(saw_hs, num_bands)
    input_bands = np.array_split(input_hs, num_bands)

    final_bands = np.empty_like(saw_bands)
    for i in range(num_bands):
        amp_multi = np.abs(saw_bands[i])*np.abs(input_bands[i])
        phase_multi = np.angle(saw_bands[i])
        final_bands[i] = amp_multi*(np.cos(phase_multi)+(np.sin(phase_multi)*1j))
        
    temp_final.append(np.ma.concatenate(final_bands).data)

In [18]:
final_wave = []
for i in range(len(temp_final)):
    final_wave.append(thinkdsp.Spectrum(hs=temp_final[i], framerate = full_input_wave.framerate).make_wave())

In [19]:
output = final_wave[0]
for i in range(1,len(final_wave)):
    output |= final_wave[i]

In [20]:
output.make_audio()